In [1]:
import graphlab as gl
import graphlab.aggregate as agg
from graphlab.toolkits.feature_engineering import NumericImputer
import numpy as np
from sklearn.decomposition import PCA
import pickle
import pandas as pd
import os
import datetime as dt

This non-commercial license of GraphLab Create for academic use is assigned to fella.belkham@u-psud.fr and will expire on December 27, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1483471899.log


In [ ]:
print ('Loading the training set ..')

columns =  ['DATE','WEEK_END', 'DAY_WE_DS', 'TPER_TEAM',
            'ASS_ASSIGNMENT', 'ASS_PARTNER', 'ASS_POLE',
            'CSPL_ABNCALLS','CSPL_MAXINQUEUE', 'CSPL_ACCEPTABLE', 'CSPL_I_ACDTIME',
            'CSPL_ABANDONNED_CALLS', 'CSPL_CALLS', 'CSPL_RECEIVED_CALLS'
           ]
url = 'train_2011_2012_2013.csv'
callsData = gl.SFrame.read_csv(url, delimiter=';', header=True, usecols= columns)

In [ ]:
print "Loading test set"
test=gl.SFrame.read_csv('submission.txt', delimiter='\t', verbose=False)
test=test['DATE', 'ASS_ASSIGNMENT']

In [ ]:
print('Splitting column DATE into columns year, month, day, hour, minute.. in the Training set')      

date_col=callsData['DATE'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.000'))
splitted_date_col=date_col.split_datetime(column_name_prefix='', limit=['year', 'month', 'day', 'hour', 'minute'])
for col in ['year', 'month', 'day', 'hour', 'minute']:
    callsData[col] = splitted_date_col[col]

print('Splitting column DATE into columns year, month, day, hour, minute.. In the Test set')      
date_col2=test['DATE'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.000'))
splitted_date_col=date_col2.split_datetime(column_name_prefix='', limit=['year', 'month', 'day', 'hour', 'minute'])
for col in ['year', 'month', 'day', 'hour', 'minute']:
    test[col] = splitted_date_col[col]

In [ ]:
calls = callsData.to_dataframe()

In [ ]:
testCalls = test.to_dataframe()

In [ ]:
gpby_cols = list(callsData[str].column_names())
gpby_cols.append('WEEK_END')
gpby_cols


In [ ]:
gpCalls = calls.groupby(['DATE','WEEK_END', 'DAY_WE_DS', 'ASS_ASSIGNMENT', 'TPER_TEAM']).sum().reset_index()

In [ ]:
np.sort(calls['CSPL_RECEIVED_CALLS'].unique())

In [ ]:
np.sort(gpCalls['CSPL_RECEIVED_CALLS'].unique())

In [ ]:
verif = gpCalls.groupby(['DATE', 'ASS_ASSIGNMENT']).count().reset_index()
verif[verif['CSPL_RECEIVED_CALLS']>1]
# should be DataFrame empty 

In [ ]:
print "Transform the categories : ASS_ASSIGNMENT to ID..."

ass=gpCalls['ASS_ASSIGNMENT'].unique()
ass_numpy=ass #.to_numpy() -- add it if sframe
gpCalls['cod_ASS_ASSIGNMENT']=gpCalls['ASS_ASSIGNMENT'].apply(lambda x: np.where(ass_numpy==x)[0][0])
testCalls['cod_ASS_ASSIGNMENT']=testCalls['ASS_ASSIGNMENT'].apply(lambda x: np.where(ass_numpy==x)[0][0])


In [ ]:
print "Adding colum, Day to Test set"
testCalls['DAY_WE_DS'] = date_col2.apply(lambda x: x.weekday())

In [ ]:
print 'Adding weekend column to Test set'
testCalls['DAY_WE'] =np.where(testCalls['DAY_WE_DS']>= 5, 1,0)

In [ ]:
print('Changing day label column into numerical in the right order (Monday=0,Thuesday=1,...)..')   
#update date col 
up_date_col=gpCalls['DATE'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.000'))
gpCalls['DAY_WE_DS'] = up_date_col.apply(lambda x: x.weekday())

In [ ]:
pickle.dump(gpCalls, open("gpCalles.obj", "wb"))

In [ ]:
pickle.dump(testCalls, open("testCalls.obj", "wb"))

In [ ]:
gpCalls.shape

In [ ]:
### adding columns

In [ ]:
# Niveau de pique par heure if not weekend
wd_niv_0 = [0, 1, 2, 3, 4, 5, 6, 7, 21, 22, 23]
wd_niv_1 = [8, 19, 20]
wd_niv_2 = [12, 13,18 ]
wd_niv_3 = [9, 14 ,15, 16 ,17]
wd_niv_4 = [10, 11]

# Niveau de pique par heure if weekend 
we_niv_0 = [0, 1, 2, 3, 4, 5, 6, 7, 8,  21, 22 ,23]
we_niv_1 = [9 ,10 ,12 ,13 ,14 ,15, 16 ,17, 18, 19, 20 ]
we_niv_2 = [11]

